# Schwarzschild black-hole immersed in a magnetic field
by Olivier Minazzoli and Maxime Wavasseur

The action of Entanled Relativiy reads as follows

<!-- $S = \int \mathrm{d}^4 x\sqrt{-g} \left( \frac{\Phi R}{2\kappa} + \sqrt{\Phi} \mathcal{L}_m \right)$ -->
$S = -\frac{1}{2}\int \mathrm{d}^4 x\sqrt{-g} \frac{\mathcal{L}_m^2}{R}$

The fields equations are

<!-- $G_{\mu\nu} = \frac{\kappa}{\sqrt{\Phi}}T_{\mu\nu} + \frac{1}{\Phi} \left( \nabla_\mu \nabla_\nu - g_{\mu\nu} \Box \right) \Phi$ -->
$G_{\mu\nu} = -\frac{R}{\mathcal{L}_m}T_{\mu\nu} + \frac{R^2}{\mathcal{L}_m^2} \left( \nabla_\mu \nabla_\nu - g_{\mu\nu} \Box \right) \frac{\mathcal{L}_m^2}{R^2}$,

and $\nabla_\sigma \left(\frac{\mathcal{L}}{R} F^{\mu \sigma}\right) = 0$,

with $\mathcal{L}_m = -\frac{1}{2} F_{\mu\nu}F^{\mu\nu}$.

The solution for the metric is
$$ds^2 = \Lambda^{\frac{20}{13}} \left[\left(1 - \frac{r_S}{r}\right)^{-1}dr^2 + r^2 d\theta^2 - \left(1 - \frac{r_S}{r}\right) dt^2 \right] + \Lambda^{-\frac{28}{13}} r^2 sin^2(\theta) d\varphi^2,$$
with
$$\Lambda = 1 + \frac{13}{48} B^2 r^2 sin^2(\theta).$$

The solution for the electromagnetic potential 4-vector is
$$A_\varphi = \frac{B}{2 \Lambda} r^2 \sin^2(\theta).$$

The solution is such that
$$-\frac{\mathcal{L}_m}{R} = \Lambda^{2/13}.$$

Below, we verify that solution.

In [1]:
version()

'SageMath version 9.5, Release Date: 2022-01-30'

In [2]:
%display latex

In [3]:
from sage.manifolds.operators import dalembertian
from sage.manifolds.operators import laplacian
from sage.manifolds.operators import grad

In [4]:
M = Manifold(4, 'M', structure='Lorentzian')
print(M)

4-dimensional Lorentzian manifold M


/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [5]:
X.<t,r,th,ph> = M.chart(r"t r:(0,+oo) th:(0,pi):\theta ph:\varphi")
r_s = var('r_s')
assume(r_s>=0)
assume(r_s<r) # focus on exterior solution
B = var('B')

Let's define two functions to verify the properties of a null complex tetrad and to calculate the Weyl scalars.

In [6]:
def tetrad_cond(Ld,Nd,Md,Mbar_d):
    for c in [Ld,Nd,Md,Mbar_d]:
        if c.tensor_type() == (1, 0):
            raise ValueError("not a covariant vector")
    L, N, Mr, Mbar = Ld.up(g), Nd.up(g), Md.up(g), Mbar_d.up(g)

    l_scalar, n_scalar, m_scalar, mbar_scalar =  (Ld.contract(0, L, 0)).expr(), (Nd.contract(0, N, 0)).expr(),\
                                    (Md.contract(0, Mr, 0)).expr(), (Mbar_d.contract(0, Mbar, 0)).expr()
    lm, mn,ln, mbarm = (Ld.contract(0, Mr, 0)).expr(), (Nd.contract(0, Mr, 0)).expr(),\
                                    (Nd.contract(0, L, 0)).expr(), (Mbar_d.contract(0, Mr, 0)).expr()
    show(LatexExpr(r'l^{\mu}l_{\mu} = '),l_scalar, LatexExpr(r'\hspace{0.5cm} n^{\mu}n_{\mu} = '),\
         n_scalar, LatexExpr(r'\hspace{0.5cm} m^{\mu}m_{\mu} = '),m_scalar, \
         LatexExpr(r'\hspace{0.5cm} \bar{m}^{\mu}\bar{m}_{\mu} = '),mbar_scalar)

    show(LatexExpr(r'l^{\mu}m_{\mu} = '),lm, LatexExpr(r'\hspace{0.5cm} m^{\mu}n_{\mu} = '),\
         mn)
    
    show(LatexExpr(r'l^{\mu}n_{\mu} = '),ln, LatexExpr(r'\hspace{0.5cm} m^{\mu}\bar{m}_{\mu} = '),mbarm)
    
    g_t = M.tensor_field(0, 2, name='g_t')
    for i in range(4):
        g_t[i,i] = -2*Ld[i]*Nd[i] + 2*Md[i]*Mbar_d[i]
        g_t[i,i] = g_t[i,i].expr().canonicalize_radical().factor().factor()
    show(LatexExpr(r'g_{ab} = (l_{a}n_{b} +l_{b}n_{a} -m_{a}\bar{m}_{b}-m_{b}\bar{m}_{a}) \Rightarrow '), g_t - g == 0)

In [7]:
def Weyl_sc(K,L,Mb,MB,C):
    
    k, l, m, mb = K.up(g), L.up(g), Mb.up(g), MB.up(g)
    P0 = k*m*k*m
    P1 = k*l*k*m
    P2 = k*m*mb*l
    P3 = l*k*l*mb
    P4 = l*mb*l*mb

    l0 = r'\Psi_{0} = W_{\alpha\beta\gamma\delta}l^{\alpha}m^{\beta}l^{\gamma}m^{\delta} = ' 
    l1 = r'\Psi_{1} = W_{\alpha\beta\gamma\delta}l^{\alpha}n^{\beta}l^{\gamma}m^{\delta} = ' 
    l2 = r'\Psi_{2} = W_{\alpha\beta\gamma\delta}l^{\alpha}m^{\beta}\bar{m}^{\gamma}n^{\delta} = ' 
    l3 = r'\Psi_{3} = W_{\alpha\beta\gamma\delta}l^{\alpha}n^{\beta}\bar{m}^{\gamma}n^{\delta} = ' 
    l4 = r'\Psi_{4} = W_{\alpha\beta\gamma\delta}n^{\alpha}\bar{m}^{\beta}n^{\gamma}\bar{m}^{\delta} = ' 

    for (l,p) in zip([l0,l1,l2,l3,l4],[P0,P1,P2,P3,P4]):
        W_sc = C['_{abcd}']*p['^{abcd}']
        show(LatexExpr(l), W_sc.expr().canonicalize_radical().factor(), \
            LatexExpr(r' \underset{B = 0}{\longrightarrow}  '), W_sc.expr().subs(B=0).canonicalize_radical().factor())

# I. Definition of the fields

In [8]:
g = M.metric()
Lambda(r,th) = 1 + 13 / 48 * B**2 * (r*sin(th))**2

g[0,0] = -(1- r_s/r) * Lambda(r,th)**(20 / 13)
g[1,1] = 1/(1- r_s/r) * Lambda(r,th)**(20 / 13)
g[2,2] = r**2 * Lambda(r,th)**(20 / 13)
g[3,3] = (r*sin(th))**2 / Lambda(r,th)**(28 / 13)

pot_vec = M.tensor_field(0,1,name='A')
pot_vec[0]=0
pot_vec[1]=0
pot_vec[2]=0
pot_vec[3]= B * (r*sin(th))**2 / (2 * Lambda(r)) 

The metric and the potential vector read

In [9]:
show(g.display())
show(pot_vec.display())

g = (13/48*B^2*r^2*sin(th)^2 + 1)^(20/13)*(r_s/r - 1) dt⊗dt - (13/48*B^2*r^2*sin(th)^2 + 1)^(20/13)/(r_s/r - 1) dr⊗dr + (13/48*B^2*r^2*sin(th)^2 + 1)^(20/13)*r^2 dth⊗dth + r^2*sin(th)^2/(13/48*B^2*r^2*sin(th)^2 + 1)^(28/13) dph⊗dph

A = 24*B*r^2*sin(th)^2/(13*B^2*r^2*sin(th)^2 + 48) dph

Let's define the covariant derivative

In [10]:
nab = g.connection() 

and the associated curvature quantities

In [11]:
Ric = g.ricci() #Ricci tensor
R = g.ricci_scalar() #Ricci scalar
G = Ric-R*g/2 #Einstein tensor

# II. Petrov classification

In [12]:
C = (g.weyl()).down(g)

Let's first define the null tetrad components

In [13]:
K = M.one_form({X.frame(): [sqrt(-g[0,0]/2), sqrt(g[1,1]/2), 0, 0]})
L = M.one_form({X.frame(): [sqrt(-g[0,0]/2), -sqrt(g[1,1]/2), 0, 0]})
Mb = M.one_form({X.frame(): [0, 0, sqrt(g[2,2]/2), i*sqrt(g[3,3]/2)]})
MB = M.one_form({X.frame(): [0, 0, sqrt(g[2,2]/2), -i*sqrt(g[3,3]/2)]})

Now we check that the tetrad components satisfy the orthogonality and normalization conditions and are related to the metric.

In [14]:
tetrad_cond(K,L,Mb,MB)

l^{\mu}l_{\mu} =  0 \hspace{0.5cm} n^{\mu}n_{\mu} =  0 \hspace{0.5cm} m^{\mu}m_{\mu} =  0 \hspace{0.5cm} \bar{m}^{\mu}\bar{m}_{\mu} =  0

l^{\mu}m_{\mu} =  0 \hspace{0.5cm} m^{\mu}n_{\mu} =  0

l^{\mu}n_{\mu} =  -1 \hspace{0.5cm} m^{\mu}\bar{m}_{\mu} =  1

g_{ab} = (l_{a}n_{b} +l_{b}n_{a} -m_{a}\bar{m}_{b}-m_{b}\bar{m}_{a}) \Rightarrow  True

In [15]:
Weyl_sc(K,L,Mb,MB,C)

\Psi_{0} = W_{\alpha\beta\gamma\delta}l^{\alpha}m^{\beta}l^{\gamma}m^{\delta} =  2304*3^(7/13)*2^(2/13)*(35*B^2*r^2*sin(th)^2 - 144)*B^2*(r - r_s)*sin(th)^2/((13*B^2*r^2*sin(th)^2 + 48)^(46/13)*r)  \underset{B = 0}{\longrightarrow}   0

\Psi_{1} = W_{\alpha\beta\gamma\delta}l^{\alpha}n^{\beta}l^{\gamma}m^{\delta} =  2304*3^(7/13)*2^(2/13)*(35*B^2*r^2*sin(th)^2 - 144)*B^2*sqrt(r - r_s)*cos(th)*sin(th)/((13*B^2*r^2*sin(th)^2 + 48)^(46/13)*sqrt(r))  \underset{B = 0}{\longrightarrow}   0

\Psi_{2} = W_{\alpha\beta\gamma\delta}l^{\alpha}m^{\beta}\bar{m}^{\gamma}n^{\delta} =  -96*3^(7/13)*2^(2/13)*(840*B^4*r^5*sin(th)^4 - 423*B^4*r^4*r_s*sin(th)^4 - 560*B^4*r^5*sin(th)^2 - 3456*B^2*r^3*sin(th)^2 + 1248*B^2*r^2*r_s*sin(th)^2 + 2304*B^2*r^3 + 2304*r_s)/((13*B^2*r^2*sin(th)^2 + 48)^(46/13)*r^3)  \underset{B = 0}{\longrightarrow}   -1/2*r_s/r^3

\Psi_{3} = W_{\alpha\beta\gamma\delta}l^{\alpha}n^{\beta}\bar{m}^{\gamma}n^{\delta} =  -2304*3^(7/13)*2^(2/13)*(35*B^2*r^2*sin(th)^2 - 144)*B^2*sqrt(r - r_s)*cos(th)*sin(th)/((13*B^2*r^2*sin(th)^2 + 48)^(46/13)*sqrt(r))  \underset{B = 0}{\longrightarrow}   0

\Psi_{4} = W_{\alpha\beta\gamma\delta}n^{\alpha}\bar{m}^{\beta}n^{\gamma}\bar{m}^{\delta} =  2304*3^(7/13)*2^(2/13)*(35*B^2*r^2*sin(th)^2 - 144)*B^2*(r - r_s)*sin(th)^2/((13*B^2*r^2*sin(th)^2 + 48)^(46/13)*r)  \underset{B = 0}{\longrightarrow}   0

### Type D in the case B=0

# III. Electromagnetic field equation

In [16]:
DF = nab(pot_vec) 

In [17]:
F=(DF['_ij']-DF['_ji'])._tensor
Fuu = F.up(g)
show(LatexExpr(r'F_{\mu\nu} ='), F.display())

F_{\mu\nu} = -2304*B*r*sin(th)^2/(169*B^4*r^4*sin(th)^4 + 1248*B^2*r^2*sin(th)^2 + 2304) dr⊗dph - 2304*B*r^2*cos(th)*sin(th)/(169*B^4*r^4*sin(th)^4 + 1248*B^2*r^2*sin(th)^2 + 2304) dth⊗dph + 2304*B*r*sin(th)^2/(169*B^4*r^4*sin(th)^4 + 1248*B^2*r^2*sin(th)^2 + 2304) dph⊗dr + 2304*B*r^2*cos(th)*sin(th)/(169*B^4*r^4*sin(th)^4 + 1248*B^2*r^2*sin(th)^2 + 2304) dph⊗dth

In [18]:
Lm = -F['_ij']*F.up(g)['^ij']/2
show(LatexExpr(r'L_{m} ='), Lm.expr().canonicalize_radical().factor())

L_{m} = 221184*3^(5/13)*2^(7/13)*(r_s*sin(th)^2 - r)*B^2/((13*B^2*r^2*sin(th)^2 + 48)^(44/13)*r)

<!-- Let's verify the Maxwell equations $\triangledown\left(\sqrt{\phi}F_{\mu\nu}\right)=0$  -->
Let's verify the Maxwell equations $\nabla_\sigma \left(\frac{\mathcal{L}}{R} F^{\mu \sigma}\right) = 0$

In [19]:
N = nab(Lm / R *F.up(g))['_i^ij']
N == 0

True

## III.1. Properties

In [20]:
show(LatexExpr(r'L_{m}\Lambda^{44/13} ='), (Lm*Lambda**(44/13)).expr().canonicalize_radical().factor())
show(LatexExpr(r'\Rightarrow L_{m} = \frac{{\left(r_{s} \sin\left({\theta}\right)^{2} - r\right)} B^{2}}{\Lambda^{44/13}r}'))
show(LatexExpr(r'\lim_{\theta \to 0} \Lambda(r,\theta) =1 \Rightarrow  \lim_{\theta \to 0} L_{m} = '), Lm.expr().subs(th=0).canonicalize_radical())

L_{m}\Lambda^{44/13} = (r_s*sin(th)^2 - r)*B^2/r

\Rightarrow L_{m} = \frac{{\left(r_{s} \sin\left({\theta}\right)^{2} - r\right)} B^{2}}{\Lambda^{44/13}r}

\lim_{\theta \to 0} \Lambda(r,\theta) =1 \Rightarrow  \lim_{\theta \to 0} L_{m} =  -B^2

We can check that $\frac{221184\times3^{5/13}\times2^{7/13}}{48^{4/13}}=1$

In [21]:
((221184*3**(5/13)*2**(7/13)/48**(44/13)).canonicalize_radical() - 1).is_zero()

True

In [22]:
epsilon_3D = M.tensor_field(0,3, antisymmetric=True, name=r'\epsilon_{c}')

epsilon_3D[1,2,3] = 1
epsilon_3D[2,3,1] = 1
epsilon_3D[3,1,2] = 1
epsilon_3D[3,2,1] = -1
epsilon_3D[2,1,3] = -1
epsilon_3D[1,3,2] = -1

epsilon_3D.display()

\epsilon_{c} = dr⊗dth⊗dph - dr⊗dph⊗dth - dth⊗dr⊗dph + dth⊗dph⊗dr + dph⊗dr⊗dth - dph⊗dth⊗dr

In [23]:
B_mag = M.tensor_field(1,0)  
for i in [1, 2, 3]:
    B_mag[i] = 1/2 * sum(epsilon_3D[i,j,k] * F[j,k] for j in [1,2,3] for k in [1,2,3])
#B_mag.apply_map(lambda f:f.substitute_function(Lambda,Lamb).simplify())
show(LatexExpr(r'B^{\mu} = '),B_mag[:])

B^{\mu} =  [0,
 -2304*B*r^2*cos(th)*sin(th)/(169*B^4*r^4*sin(th)^4 + 1248*B^2*r^2*sin(th)^2 + 2304),
 2304*B*r*sin(th)^2/(169*B^4*r^4*sin(th)^4 + 1248*B^2*r^2*sin(th)^2 + 2304),
 0]

In [24]:
B_mag[1]== -B*r**2*sin(th)*cos(th)/Lambda**2

True

In [25]:
B_mag[2]== B*r*sin(th)**2/Lambda**2

True

# IV. The metric field equation

Let's define the stress-energy tensor $T_{\mu\nu}=2\left(F_{\rho\mu}F^{\rho}_{\hspace{0.2cm}\nu}-\frac{1}{4}g_{\mu\nu}F^{2}\right)$ 

In [26]:
T = 2*(F.up(g,1)['_i^j']*F['_kj'] + g*Lm/2)

In [27]:
T_scalar = g.inverse()['^ab'] * T['_ab']

In [28]:
T_scalar == 0

True

In [29]:
S = (nab(nab(Lm**2 / R**2 )) - g*(Lm**2 / R**2).dalembertian()) * R**2 / Lm**2 

And the field equation

In [30]:
G == - R / Lm * T + S

True

# V. The scalar-field form $-\frac{\mathcal{L}_m}{R} = \Lambda^{2/13}$

In [31]:
Lambda**(2/13) + Lm/R == 0

True

## V.1. Properties

In [32]:
show(LatexExpr(r'R ='), R.expr().canonicalize_radical().factor())
show(LatexExpr(r'R\Lambda^{46/13} ='), (R*Lambda**(46/13)).expr().canonicalize_radical().factor())
show(LatexExpr(r'\Rightarrow R = \frac{{\left(r_{s} \sin\left({\theta}\right)^{2} - r\right)} B^{2}}{\Lambda^{46/13}r} = L_{m} \Lambda^{2/13}'))
show(LatexExpr(r'\lim_{\theta \to 0} \Lambda(r,\theta) =1 \Rightarrow  \lim_{\theta \to 0} R = '), R.expr().subs(th=0).canonicalize_radical())

R = -442368*3^(7/13)*2^(2/13)*(r_s*sin(th)^2 - r)*B^2/((13*B^2*r^2*sin(th)^2 + 48)^(46/13)*r)

R\Lambda^{46/13} = -(r_s*sin(th)^2 - r)*B^2/r

\Rightarrow R = \frac{{\left(r_{s} \sin\left({\theta}\right)^{2} - r\right)} B^{2}}{\Lambda^{46/13}r} = L_{m} \Lambda^{2/13}

\lim_{\theta \to 0} \Lambda(r,\theta) =1 \Rightarrow  \lim_{\theta \to 0} R =  B^2

We can check that $\frac{442368\times3^{7/13}\times2^{2/13}}{48^{46/13}}=1$

In [33]:
((442368*3**(7/13)*2**(2/13) / 48**(46/13)).canonicalize_radical() - 1).is_zero()

True